In [17]:
import pandas as pd
import os
import requests
import subprocess
from faster_whisper import WhisperModel
from tqdm import tqdm

In [18]:
# --------------------------------------------------
# CONFIG
# --------------------------------------------------

CSV_PATH = "../audio_links.csv"
DOWNLOAD_DIR = "../audio/downloaded"
MODEL_PATH = r"C:\whisper_models\faster-whisper-small"

os.makedirs(DOWNLOAD_DIR, exist_ok=True)
os.makedirs("../output", exist_ok=True)


In [19]:
# --------------------------------------------------
# LOAD CSV
# --------------------------------------------------

cols = ["Podcast_Name", "Episode_Name", "Audio_URL", "Source", "Index"]
df = pd.read_csv(CSV_PATH, header=None, names=cols)


In [ ]:
# --------------------------------------------------
# BATCH SELECTION (ONLY UNCOMMENT ONE AT A TIME)
# Total rows: 0–384
# Batch size: 50
# --------------------------------------------------

#Batch 0: rows 0-1
# df = df.iloc[0:2]
# OUTPUT_CSV = "../output/raw_transcripts_0_1.csv"

#Batch 0.1: rows 2
df = df.iloc[2:3]
OUTPUT_CSV = "../output/raw_transcripts_2.csv"


# Batch 1: rows 3–50
# df = df.iloc[3:51]
# OUTPUT_CSV = "../output/raw_transcripts_3_50.csv"

# Batch 2: rows 51–99
# df = df.iloc[51:100]
# OUTPUT_CSV = "../output/raw_transcripts_51_99.csv"

# Batch 3: rows 100–149
# df = df.iloc[100:150]
# OUTPUT_CSV = "../output/raw_transcripts_100_149.csv"

# Batch 4: rows 150–199
# df = df.iloc[150:200]
# OUTPUT_CSV = "../output/raw_transcripts_150_199.csv"

# Batch 5: rows 200–249
# df = df.iloc[200:250]
# OUTPUT_CSV = "../output/raw_transcripts_200_249.csv"

# Batch 6: rows 250–299
# df = df.iloc[250:300]
# OUTPUT_CSV = "../output/raw_transcripts_250_299.csv"

# Batch 7: rows 300–349
# df = df.iloc[300:350]
# OUTPUT_CSV = "../output/raw_transcripts_300_349.csv"

# Batch 8: rows 350–384 (last batch)
# df = df.iloc[350:385]
# OUTPUT_CSV = "../output/raw_transcripts_350_384.csv"

In [21]:
# --------------------------------------------------
# SAFETY CHECKS
# --------------------------------------------------

print(f"Processing rows {df.index.min()} to {df.index.max()}")
print(f"Total files in this batch: {len(df)}")
print(f"Output file: {OUTPUT_CSV}")

if os.path.exists(OUTPUT_CSV):
    raise RuntimeError(
        f"{OUTPUT_CSV} already exists. Stop to avoid overwrite."
    )

Processing rows 2 to 2
Total files in this batch: 1
Output file: ../output/raw_transcripts_2.csv


In [22]:
model = WhisperModel(
    MODEL_PATH,
    device="cpu",
    compute_type="int8",   # 🔥 fastest on CPU
    cpu_threads=12,        # 🔥 use all cores
    num_workers=4          # 🔥 parallel decoding
)

print("Whisper running on CPU (optimized, stable mode)")

Whisper running on CPU (optimized, stable mode)


In [23]:
# --------------------------------------------------
# PROCESS AUDIO FILES
# --------------------------------------------------

results = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    try:
        audio_url = row["Audio_URL"]
        index_id = row["Index"]

        mp3_path = f"{DOWNLOAD_DIR}/{index_id}.mp3"
        wav_path = f"{DOWNLOAD_DIR}/{index_id}.wav"

        # -------- Download audio --------
        if not os.path.exists(mp3_path):
            r = requests.get(audio_url, timeout=30)
            r.raise_for_status()
            with open(mp3_path, "wb") as f:
                f.write(r.content)

        # -------- Convert to WAV --------
        subprocess.run(
            [
                "ffmpeg", "-y",
                "-i", mp3_path,
                "-ac", "1",
                "-ar", "16000",
                wav_path
            ],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL,
            check=True
        )

        # -------- Transcribe (RAW ONLY) --------
        segments, info = model.transcribe(wav_path)
        segments = list(segments)

        raw_text = " ".join(seg.text.strip() for seg in segments)

        # -------- Save result --------
        results.append({
            "Podcast_Name": row["Podcast_Name"],
            "Episode_Name": row["Episode_Name"],
            "Audio_URL": audio_url,
            "Index": index_id,
            "Language": info.language,
            "Raw_Transcript": raw_text
        })

    except Exception as e:
        print(f"❌ Failed for Index {row['Index']}: {e}")
        continue

100%|██████████| 1/1 [05:59<00:00, 359.43s/it]


In [24]:
# --------------------------------------------------
# SAVE OUTPUT
# --------------------------------------------------

output_df = pd.DataFrame(results)
output_df.to_csv(OUTPUT_CSV, index=False)

print("\n✅ Batch completed successfully")
print(f"Saved to: {OUTPUT_CSV}")


✅ Batch completed successfully
Saved to: ../output/raw_transcripts_2.csv
